In [3]:
# default_exp algo.dl.transformers

%reload_ext autoreload
%autoreload 2

# algo-dl-transformers
https://github.com/huggingface/transformers

# Preprocessing data
https://huggingface.co/transformers/preprocessing.html

In this tutorial, we’ll explore how to preprocess your data using 🤗 Transformers. The main tool for this is what we call a tokenizer. You can build one using the tokenizer class associated to the model you would like to use, or directly with the AutoTokenizer class.

As we saw in the quicktour, 

    the tokenizer will first split a given text in words (or part of words, punctuation symbols, etc.) usually called tokens. 
    Then it will convert those tokens into numbers, to be able to build a tensor out of them and feed them to the model. 
    It will also add any additional inputs the model might expect to work properly.

## Note

If you plan on using a pretrained model, it’s important to use the associated pretrained tokenizer: it will split the text you give it in tokens the same way for the pretraining corpus, and it will use the same correspondence token to index (that we usually call a vocab) as during pretraining.

To automatically download the vocab used during pretraining or fine-tuning a given model, you can use the from_pretrained() method:

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')

# Tokenizer summary
https://huggingface.co/transformers/tokenizer_summary.html#introduction-to-tokenization

In this page, we will have a closer look at tokenization. As we saw in the preprocessing tutorial, 

    tokenizing a text is splitting it into words or subwords, which then are converted to ids. 
    
The second part is pretty straightforward, here we will focus on the first part. More specifically, we will look at the 
## three main different kinds of tokenizers used in Transformers: 
* Byte-Pair Encoding (BPE), 
* WordPiece and 
* SentencePiece, 

and provide examples of models using each of those.

Note that on each model page, you can look at the documentation of the associated tokenizer to know which of those algorithms the pretrained model used. For instance, if we look at BertTokenizer, we can see it’s using WordPiece.

Introduction to tokenization
Splitting a text in smaller chunks is a task that’s harder than it looks, and there are multiple ways of doing it. For instance, let’s look at the sentence “Don’t you love 🤗 Transformers? We sure do.” A first simple way of tokenizing this text is just to split it by spaces, which would give:

## Tokenizer
https://huggingface.co/transformers/main_classes/tokenizer.html

A tokenizer is in charge of preparing the inputs for a model. 

The library comprise tokenizers for all the models. 

Most of the tokenizers are available in two flavors: 
* a full python implementation and 
* a “Fast” implementation based on the Rust library tokenizers. 

The “Fast” implementations allows 
* (1) a significant speed-up in particular when doing batched tokenization and 
* (2) additional methods to map between the original string (character and words) and the token space (e.g. getting the index of the token comprising a given character or the span of characters corresponding to a given token).   
Currently no “Fast” implementation is available for the SentencePiece-based tokenizers (for T5, ALBERT, CamemBERT, XLMRoBERTa and XLNet models).

The base classes PreTrainedTokenizer and PreTrainedTokenizerFast implements the common methods for encoding string inputs in model inputs (see below) and instantiating/saving python and “Fast” tokenizers either from a local file or directory or from a pretrained tokenizer provided by the library (downloaded from HuggingFace’s AWS S3 repository).

PreTrainedTokenizer and PreTrainedTokenizerFast thus implements the main methods for using all the tokenizers:

* tokenizing (spliting strings in sub-word token strings), converting tokens strings to ids and back, and encoding/decoding (i.e. tokenizing + convert to integers),

* adding new tokens to the vocabulary in a way that is independant of the underlying structure (BPE, SentencePiece…),

* managing special tokens like mask, beginning-of-sentence, etc tokens (adding them, assigning them to attributes in the tokenizer for easy access and making sure they are not split during tokenization)

`BatchEncoding` holds the output of the tokenizer’s encoding methods (`__call__`, encode_plus and batch_encode_plus) and is derived from a Python dictionary. 
* When the tokenizer is a pure python tokenizer, this class behave just like a standard python dictionary and hold the various model inputs computed by these methodes (input_ids, attention_mask…). 
* When the tokenizer is a “Fast” tokenizer (i.e. backed by HuggingFace tokenizers library), this class provides in addition several advanced alignement methods which can be used to map between the original string (character and words) and the token space (e.g. getting the index of the token comprising a given character or the span of characters corresponding to a given token).

## install

In [1]:
# !pip install transformers
!pip freeze | grep transformers

transformers==3.0.2


In [2]:
import torch
from transformers import *

I0716 19:16:41.659997 140735727772544 file_utils.py:39] PyTorch version 1.5.1 available.
I0716 19:16:46.939681 140735727772544 file_utils.py:55] TensorFlow version 2.2.0 available.


ImportError: dlopen(/Users/luoyonggui/anaconda3/lib/python3.7/site-packages/tokenizers/tokenizers.cpython-37m-darwin.so, 2): Symbol not found: ____chkstk_darwin
  Referenced from: /Users/luoyonggui/anaconda3/lib/python3.7/site-packages/tokenizers/tokenizers.cpython-37m-darwin.so (which was built for Mac OS X 10.15)
  Expected in: /usr/lib/libSystem.B.dylib
 in /Users/luoyonggui/anaconda3/lib/python3.7/site-packages/tokenizers/tokenizers.cpython-37m-darwin.so

In [1]:


# Transformers has a unified API
# for 10 transformer architectures and 30 pretrained weights.
#          Model          | Tokenizer          | Pretrained weights shortcut
MODELS = [(BertModel,       BertTokenizer,       'bert-base-uncased'),
          (OpenAIGPTModel,  OpenAIGPTTokenizer,  'openai-gpt'),
          (GPT2Model,       GPT2Tokenizer,       'gpt2'),
          (CTRLModel,       CTRLTokenizer,       'ctrl'),
          (TransfoXLModel,  TransfoXLTokenizer,  'transfo-xl-wt103'),
          (XLNetModel,      XLNetTokenizer,      'xlnet-base-cased'),
          (XLMModel,        XLMTokenizer,        'xlm-mlm-enfr-1024'),
          (DistilBertModel, DistilBertTokenizer, 'distilbert-base-cased'),
          (RobertaModel,    RobertaTokenizer,    'roberta-base'),
          (XLMRobertaModel, XLMRobertaTokenizer, 'xlm-roberta-base'),
         ]

# To use TensorFlow 2.0 versions of the models, simply prefix the class names with 'TF', e.g. `TFRobertaModel` is the TF 2.0 counterpart of the PyTorch model `RobertaModel`

# Let's encode some text in a sequence of hidden-states using each model:
for model_class, tokenizer_class, pretrained_weights in MODELS:
    # Load pretrained model/tokenizer
    tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
    model = model_class.from_pretrained(pretrained_weights)

    # Encode text
    input_ids = torch.tensor([tokenizer.encode("Here is some text to encode", add_special_tokens=True)])  # Add special tokens takes care of adding [CLS], [SEP], <s>... tokens in the right way for each model.
    with torch.no_grad():
        last_hidden_states = model(input_ids)[0]  # Models outputs are now tuples

# Each architecture is provided with several class for fine-tuning on down-stream tasks, e.g.
BERT_MODEL_CLASSES = [BertModel, BertForPreTraining, BertForMaskedLM, BertForNextSentencePrediction,
                      BertForSequenceClassification, BertForTokenClassification, BertForQuestionAnswering]

# All the classes for an architecture can be initiated from pretrained weights for this architecture
# Note that additional weights added for fine-tuning are only initialized
# and need to be trained on the down-stream task
pretrained_weights = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(pretrained_weights)
for model_class in BERT_MODEL_CLASSES:
    # Load pretrained model/tokenizer
    model = model_class.from_pretrained(pretrained_weights)

    # Models can return full list of hidden-states & attentions weights at each layer
    model = model_class.from_pretrained(pretrained_weights,
                                        output_hidden_states=True,
                                        output_attentions=True)
    input_ids = torch.tensor([tokenizer.encode("Let's see all hidden-states and attentions on this text")])
    all_hidden_states, all_attentions = model(input_ids)[-2:]

    # Models are compatible with Torchscript
    model = model_class.from_pretrained(pretrained_weights, torchscript=True)
    traced_model = torch.jit.trace(model, (input_ids,))

    # Simple serialization for models and tokenizers
    model.save_pretrained('./directory/to/save/')  # save
    model = model_class.from_pretrained('./directory/to/save/')  # re-load
    tokenizer.save_pretrained('./directory/to/save/')  # save
    tokenizer = BertTokenizer.from_pretrained('./directory/to/save/')  # re-load

    # SOTA examples for GLUE, SQUAD, text generation...

In [ ]:
from nbdev.export import notebook2script
notebook2script('.ipynb')